# Exercise 6.2: Two masses connected by strings. 

Two masses 1 and 2, of  weights $W_1, W_2$, respectively, are hung from three pieces of string with lengths $L_1, L_2, L_3$ and a horizontal bar of length $L$. 

Using $N$-dimensional Newton-Raphson searching, find the angles $\theta_1$, $\theta_2$ 
and $\theta_3$ and the tensions exerted by the strings $T_1$, $T_2$, $T_3$. 

Use the values: $W_1 = 10$ N, $W_2 = 20$ N, $(L_1, L_2, L_3)=(3,4,4)$ m and $L=8$ m. 


Solution:

Let's begin by creating a Jacobian function that yields an $N\times N$ NumPy array of the partial derivatives as requested. We will then use NumPy's linalg package to solve the linear equation $\mathbf{J} \Delta \vec{x} = -\vec{f}$ to move through the guesses. The structure of the Newton-Raphson searching should otherwise be very similar to the one-dimensional case. 

In [12]:
import numpy as np
from numpy import linalg 

# modify from Chapter 4: 
# define a function for the central-difference derivative.
# x is now a NumPy array
# i is the component of the function (f_i) to be differentiated
# j is the component of the vector x to differentiate by (x_j)
def dfdt_CD_ND(funcvector, i, j, x, h): 
    """Calculates the central-difference partial derivative of a function func at vector x, in the j-th direction, with step size dx"""
    N = len(x) # get the length of the input array
    # increment only the j-th element by an amount dx
    dx = np.zeros(N) 
    dx[j] = h 
    return (funcvector(x+dx/2)[i] - funcvector(x-dx/2)[i])/h

# A higher-order function that calculates the Jacobian matrix for the N-dimensional vector of functions f_i:
# we will use a central-difference approximation to the derivatives, and a parameter h = 1E-5 for all directions 
def Jacobian(funcvector, x, h): # the input should be a NumPy array of functions, each of which is a function of N variables (x_i)
    """Calculate the Jacobian of the function vector as an NxN matrix using the central-difference approximation to the derivatives"""
    N = len(funcvector(x)) # get the number of dimensions
    output = np.zeros((N,N)) #  the output is an NxN NumPy array
    for i in range(N): # loop over the functions f_i
        for j in range(N): # loop over variables x_j:
            dfi_dxj = dfdt_CD_ND(funcvector, i,j,x,h)
            output[i][j] = dfi_dxj[0] # set this to the correct element of the Jacobian   
    return output 

We ought to check that this indeed works as intendend! E.g. let's get the result for a *linear* function, for which the Jacobian can be trivially calculated! 

In [13]:
# CHECK Jacobian with linear functions:
def fi_check_linear(x):
    funcmatrix = np.array([[2*x[0] + x[1] - 13], [ x[0] + x[1] - 9 ]])
    return funcmatrix

# Perform check:
h=1E-5 # step length for derivatives
x0 = [1, 2] # point for evaluation derivatives (this will be irrelevant since the functions are linear)
print(Jacobian(fi_check_linear, x0, h))

[2.]
[1.]
[1.]
[1.]
[[2. 1.]
 [1. 1.]]


The above is as expected for linear functions. Let's try it out with non-linear functions as well. 

In [3]:
# CHECK Jacobian with non-linear functions:
def fi_check_nonlinear(x):
    funcmatrix = np.array([[x[0]**2 + x[1] - 21], [ x[0] + x[1]**2 - 29 ]])
    return funcmatrix

# Perform check:
h=1E-5 # step length for derivatives
x0 = [1, 2] # point for evaluation derivatives
print(Jacobian(fi_check_nonlinear, x0, h))

[[2. 1.]
 [1. 4.]]


This also works as intended! We have a functioning Jacobian, so let's proceed with the implementation of the Newton-Raphson!

We will use the one-dimensional function as a basis (Example 6.2)

In [4]:
import numpy as np
from numpy import linalg 

# The N-D Newton-Raphson algorithm: 
# x0 is the initial guess, it should have the same dimensions as the number of variables! 
# Nmax is the number of evaluations
# prec is the required precision
# dx is the distance over which to take the central-difference derivative (not the same as the step size!)
def NewtonRaphsonND(func, x0, Nmax, prec, h): 
    """Function that implements the N-Dimensional Newton-Raphson algorithm for root finding"""
    # perform check that the number of dimensions is the same for the function and variables
    N = len(func(x0)) # get the number of dimensions
    if N != len(x0):
         raise Exception("The length of the function vector is not the same as the number of unknowns")
    n = 0 # the number of steps taken
    val = 1E99*np.ones(N) # the value of the equations, initialize to a large number
    roots = np.nan*np.ones(N) # initialize the roots to "not a number"
    for nn in range(Nmax): # loop runs up to the max number of evals, or up to the point where the precision is reached
        # get the values of the function at x0:
        minus_f = -func(x0)
        # check whether the required precision has been reached for *each* value:
        if np.all(np.abs(minus_f) < prec):
            n = nn # save the number of steps taken 
            print('Newton-Raphson Precision reached! Exiting')
            break # exit the loop nn
        # Get the Jacobian (J):
        J = Jacobian(func, x0, h)
        # calculate the step vector Dx using linear algebra (see Chapter 6, "NumPy's linalg Package" section)
        Dx = linalg.solve(J, minus_f).reshape(N) # turn this into the right shape (column vector)
        # update the guess and the value of the equation:
        x0 = np.add(x0, Dx)
    roots = x0
    return roots, n

Let's now check our N-D Newton-Raphson with the linear and non-linear check functions for which we know the solutions!

In [5]:
# CHECK the linear function: 
Nmax = 1000
prec = 1E-6
h = 1E-5
x0 = np.array([0,0])
roots, niterations = NewtonRaphsonND(fi_check_linear, x0, Nmax, prec, h)
print('The roots of the linear equations are=', roots)

# CHECK the non-linear function:
roots, niterations = NewtonRaphsonND(fi_check_nonlinear, x0, Nmax, prec, h)
print('The roots of the non-linear equations are=', roots)

Newton-Raphson Precision reached! Exiting
The roots of the linear equations are= [4. 5.]
Newton-Raphson Precision reached! Exiting
The roots of the non-linear equations are= [4.00000012 4.99999999]


Both are correct! Let's now proceed to solve our problem of two masses suspended by strings. 

We first need to write down the equations in the correct form and then pick a good starting guess. 

In [6]:
import numpy as np

# Our equation for the two masses on the string
# start with the two length constraints,
# then the equilibrium conditions
# and then the trigonometric identities
# x=(sth1, sth2, sth3, ct1, ct2, ct3, T1, T2, T3) corresponding to:
# x=(x[0], x[1], x[2], ..., x[8])
L1 = 3
L2 = 4
L3 = 4
W1 = 10
W2 = 20
L = 8
def fi_twomasses(x):
    """The nonlinear coupled equations for two masses on a string"""
    funcmatrix = np.array([
        [L1*x[3] + L2*x[4] + L3*x[5] - L], 
        [L1*x[0] + L2*x[1] - L3*x[2]],
        [x[6]*x[0] - x[7]*x[1] - W1],
        [x[6]*x[3] - x[7]*x[4]],
        [x[7]*x[1] + x[8]*x[2] - W2],
        [x[7]*x[4] - x[8]*x[5]],
        [x[0]**2 + x[3]**2 - 1],
        [x[1]**2 + x[4]**2 - 1],
        [x[2]**2 + x[5]**2 - 1]])
    return funcmatrix

In [7]:
import numpy as np 
# CHECK the linear function: 
Nmax = 1000
prec = 1E-6
h = 1E-5
x0 = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1, 1, 1])
roots, niterations = NewtonRaphsonND(fi_twomasses, x0, Nmax, prec, h)
print('The roots of the two mass equations are=', roots)
print('Reached after', niterations, 'iterations')

Newton-Raphson Precision reached! Exiting
The roots of the two mass equations are= [ 0.76100269  0.26495381  0.83570583  0.64874872  0.9642611   0.54917735
 17.16020978 11.54527968 20.27152804]
Reached after 8 iterations


Let's perform some consistency checks! 

In [8]:
# Do the angles satisfy the trig identities? 
# x=(sth1, sth2, sth3, ct1, ct2, ct3, T1, T2, T3) corresponding to:
# x=(x[0], x[1], x[2], ..., x[8])
print('sth1**2 + ct1**2=', roots[0]**2 + roots[3]**2)
print('sth2**2 + ct2**2=', roots[1]**2 + roots[4]**2)
print('sth3**2 + ct3**2=', roots[2]**2 + roots[5]**2)

sth1**2 + ct1**2= 1.0000000000008653
sth2**2 + ct2**2= 1.0000000000004041
sth3**2 + ct3**2= 1.000000000000512


They all seem correct! 
The final solutions are then:

In [9]:
theta1 = np.arcsin(roots[0])/2/np.pi * 360
theta2 = np.arcsin(roots[1])/2/np.pi * 360
theta3 = np.arcsin(roots[2])/2/np.pi * 360
print('theta_1,2,3=', theta1, theta2, theta3, 'deg')

theta_1,2,3= 49.55267292072046 15.364208216019374 56.68940565509829 deg


In [10]:
T1 = roots[6]
T2 = roots[7]
T3 = roots[8]
print('T_1,2,3=', T1, T2, T3,'N')

T_1,2,3= 17.160209784580474 11.545279684311419 20.271528044627086 N


In [11]:
# Use SciPy's root finder as a test:
from scipy import optimize
import numpy as np

# Our equation for the two masses on the string
# start with the two length constraints,
# then the equilibrium conditions
# and then the trigonometric identities
# x=(sth1, sth2, sth3, ct1, ct2, ct3, T1, T2, T3) corresponding to:
# x=(x[0], x[1], x[2], ..., x[8])
L1 = 3
L2 = 4
L3 = 4
W1 = 10
W2 = 20
L = 8
def fi_twomasses(x):
    """The nonlinear coupled equations for two masses on a string"""
    funcmatrix = np.array([
        [L1*x[3] + L2*x[4] + L3*x[5] - L], 
        [L1*x[0] + L2*x[1] - L3*x[2]],
        [x[6]*x[0] - x[7]*x[1] - W1],
        [x[6]*x[3] - x[7]*x[4]],
        [x[7]*x[1] + x[8]*x[2] - W2],
        [x[7]*x[4] - x[8]*x[5]],
        [x[0]**2 + x[3]**2 - 1],
        [x[1]**2 + x[4]**2 - 1],
        [x[2]**2 + x[5]**2 - 1]])
    return np.ravel( funcmatrix[ : , 0] ) # necessary to convert to the right format for SciPy's root finder
root = optimize.root(fi_twomasses, x0)
print(root.x)

[ 0.76100269  0.26495381  0.83570583  0.64874872  0.9642611   0.54917735
 17.16020978 11.54527968 20.27152804]
